In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from qubic.lib.Qgps import GPSCalsource, GPSAntenna
 
%matplotlib inline

In [ ]:
### Build GPS data file path
data_path = "calsource_orientation.dat"

### Define the distance between the two antennas
distance_between_antennas = 1.3

In [ ]:
### Build the GPSAntenna instance
gps_antenna = GPSAntenna(data_path, distance_between_antennas)

In [ ]:
date = np.array([dt.datetime(year=2024, month=12, day=12, hour=8, minute=55, second=0)])
index_ini = gps_antenna.get_observation_indices(gps_antenna._datetime, date)[0]
print('Initial index = ', index_ini)
print('size observation times : ', len(gps_antenna._datetime))

In [ ]:
gps_antenna.plot_gps_data(index_start=index_ini)

In [ ]:
array_hours = np.array([8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11])
array_minutes_begins = np.array([55, 00, 6, 12, 19, 25, 32, 41, 46, 52, 58, 4, 10, 19, 27, 34, 40, 46, 53, 1, 7, 13, 20, 25])

array_minutes_ending = array_minutes_begins.copy() - 1
array_minutes_ending[1] = 0

array_seconds_begins = np.ones(array_hours.shape, dtype=int) * 59
array_seconds_ending = np.zeros(array_hours.shape, dtype=int)

array_datetime = np.array([])
array_index = np.array([], dtype=int)
for index in range(array_hours.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=array_hours[index], minute=array_minutes_begins[index], second=array_seconds_begins[index]))
for index in range(1, array_hours.shape[0]):
    array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=array_hours[index], minute=array_minutes_ending[index], second=array_seconds_ending[index]))
array_datetime = np.append(array_datetime, dt.datetime(year=2024, month=12, day=12, hour=11, minute=33, second=0))

for index in range(array_datetime.shape[0]):
    array_index = np.append(array_index, gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([array_datetime[index]])))

In [ ]:
array_index.shape

In [ ]:
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpN[index_ini:], color = color_a, label = 'North component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpE[index_ini:], color = color_b, label = 'East component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpD[index_ini:], color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:], color = color_r, label = 'Roll angle')
ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.yaw[index_ini:], color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

for obs_index in array_index:
    ax1.vlines(gps_antenna._datetime[obs_index], 3.5, -1.5, 'grey', linestyles='--')
ax1.vlines(gps_antenna._datetime[array_index[6]], 3.5, -1.5, 'r', linestyles='--')
ax1.vlines(gps_antenna._datetime[array_index[6 + int(array_hours.shape[0])]], 3.5, -1.5, 'r', linestyles='--')

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend()
plt.show()

In [ ]:
# Function to remove the movement between each configurations
def only_data(array, indices):
    data_array = np.array([])
    
    for i in range(int(indices.shape[0]/2)):
        data_array = np.append(data_array, array[indices[i]:indices[i+int(indices.shape[0]/2)]])
    
    return data_array

In [ ]:
data_rpN = only_data(gps_antenna.rpN, array_index)
data_rpE = only_data(gps_antenna.rpE, array_index)
data_rpD = only_data(gps_antenna.rpD, array_index)
data_roll = only_data(gps_antenna.roll, array_index)
data_yaw = only_data(gps_antenna.yaw, array_index)

In [ ]:
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(data_rpN, color = color_a, label = 'North component')
ax1.plot(data_rpE, color = color_b, label = 'East component')
ax1.plot(data_rpD, color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(data_roll, color = color_r, label = 'Roll angle')
ax2.plot(data_yaw, color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend()
plt.show()

In [ ]:
# Function to compute the standard deviation avoiding moments when I changed the experimental configuration
def only_data_std(array, indices):
    data_array = np.array([])
    
    for i in range(int(indices.shape[0]/2)):
        data_array = np.append(data_array, np.std(array[indices[i]:indices[i+int(indices.shape[0]/2)]]))
    
    return data_array

In [ ]:
# Multiply by 100 to convert from m to cm
std_rpN = only_data_std(gps_antenna.rpN, array_index) * 100
std_rpE = only_data_std(gps_antenna.rpE, array_index) * 100
std_rpD = only_data_std(gps_antenna.rpD, array_index) * 100
std_roll = only_data_std(gps_antenna.roll, array_index) * 100
std_yaw = only_data_std(gps_antenna.yaw, array_index) * 100

In [ ]:
distance_base_antenna1 = np.array([20, 20, 20, 20, 20, 100, 100, 100, 100, 200, 200, 200, 200])
distance_base_antenna2 = np.array([20, 50, 100, 200, 300, 300, 200, 100, 50, 50, 100, 200, 300])

distance_base_antenna2_bis = np.array([300, 300, 300, 200, 200, 200, 200, 100, 100, 100, 100])
distance_antenna1_antenna2 = np.array([50, 100, 200, 50, 100, 200, 300, 50, 100, 200, 300])

In [ ]:
plt.plot(std_rpN, label = 'std_rpN')
plt.plot(std_rpE, label = 'std_rpE')
plt.plot(std_rpD, label = 'std_rpD')
plt.plot(std_roll, label = 'std_roll')
plt.plot(std_yaw, label = 'std_yaw')
plt.vlines(distance_base_antenna1.shape[0]+0.5, 0, 3, color = 'black', linestyle = '--')
plt.legend()
plt.xlabel('Experimental configurations index')
plt.ylabel('Standard deviation')
plt.title('Standard deviation of the GPS data for all experimental configurations')

In [ ]:
print(distance_base_antenna2.shape, distance_base_antenna2_bis.shape)
print(std_roll.shape)

In [ ]:
def plot_std_exp1(array):
    plt.figure()
    for _, dist in enumerate(np.unique(distance_base_antenna1)):
        mask = distance_base_antenna1 == dist
        plt.scatter(distance_base_antenna2[mask], array[:distance_base_antenna2.shape[0]][mask], label=f'Distance Base-Antenna 1: {dist} cm')
    plt.legend()
    plt.xlabel('Distance Base-Antenna 2 (cm)')
    plt.ylabel('Standard deviation (cm)')

In [ ]:
plot_std_exp1(std_rpN)
plt.title('Standard deviation of the North component of antenna 1')
plt.show()
plot_std_exp1(std_rpE)
plt.title('Standard deviation of the East component of antenna 1')
plt.show()
plot_std_exp1(std_rpD)
plt.title('Standard deviation of the Up component of antenna 1')
plt.show()
plot_std_exp1(std_roll)
plt.title('Standard deviation of the roll angle of antenna 1')
plt.show()
plot_std_exp1(std_yaw)
plt.title('Standard deviation of the yaw angle of antenna 1')
plt.show()

In [ ]:
def plot_std_exp2(array):
    plt.figure()
    for _, dist in enumerate(np.unique(distance_base_antenna2_bis)):
        mask = distance_base_antenna2_bis == dist
        plt.scatter(distance_antenna1_antenna2[mask], array[distance_base_antenna2.shape[0]:][mask], label=f'Distance Base - Antenna 2: {dist} cm')
    plt.legend()
    plt.xlabel('Distance Base-Antenna 2 (cm)')
    plt.ylabel('Standard deviation (cm)')

In [ ]:
plot_std_exp2(std_rpN)
plt.title('Standard deviation of the North component of antenna 1')
plt.show()
plot_std_exp2(std_rpE)
plt.title('Standard deviation of the East component of antenna 1')
plt.show()
plot_std_exp2(std_rpD)
plt.title('Standard deviation of the Up component of antenna 1')
plt.show()
plot_std_exp2(std_roll)
plt.title('Standard deviation of the roll angle of antenna 1')
plt.show()
plot_std_exp2(std_yaw)
plt.title('Standard deviation of the yaw angle of antenna 1')
plt.show()

# Incertainty on antenna 1 position

$\Delta r = \sqrt{(\frac{x}{r}\Delta x)^{2} + (\frac{y}{r}\Delta y)^{2} + (\frac{z}{r}\Delta z)^{2}}$

In [ ]:
delta_r = np.array([])

for i in range(int(array_index.shape[0]/2)):
    data_array_north = np.mean(gps_antenna.rpN[array_index[i]:array_index[i+int(array_index.shape[0]/2)]] )
    data_array_east = np.mean(gps_antenna.rpE[array_index[i]:array_index[i+int(array_index.shape[0]/2)]] )
    data_array_down = np.mean(gps_antenna.rpD[array_index[i]:array_index[i+int(array_index.shape[0]/2)]])

    # Don't forget to convert from m to cm
    std_north = np.std(gps_antenna.rpN[array_index[i]:array_index[i+int(array_index.shape[0]/2)]]) * 100
    std_east = np.std(gps_antenna.rpE[array_index[i]:array_index[i+int(array_index.shape[0]/2)]]) * 100
    std_down = np.std(gps_antenna.rpD[array_index[i]:array_index[i+int(array_index.shape[0]/2)]]) * 100

    r = np.sqrt(data_array_north**2 + data_array_east**2 + data_array_down**2)

    delta_r = np.append(delta_r, np.sqrt((data_array_north * std_north / r)**2 + 
                                         (data_array_east * std_east / r)**2 + 
                                         (data_array_down * std_down / r)**2))
print(delta_r.shape)

In [ ]:
plt.figure()
for _, dist in enumerate(np.unique(distance_base_antenna1)):
    mask = distance_base_antenna1 == dist
    plt.scatter(distance_base_antenna2[mask], delta_r[:distance_base_antenna2.shape[0]][mask], label=f'Distance Base-Antenna 1: {dist} cm')
plt.legend()
plt.ylabel('Relative error (cm)')
plt.xlabel('Distance Base-Antenna 2 (cm)')
plt.title('Relative error on the position of the antenna 1 during experiment 1')

In [ ]:
plt.figure()
for _, dist in enumerate(np.unique(distance_base_antenna2_bis)):
    mask = distance_base_antenna2_bis == dist
    plt.scatter(distance_antenna1_antenna2[mask], delta_r[distance_base_antenna2.shape[0]:][mask], label=f'Distance Base - Antenna 2: {dist} cm')
plt.legend()
plt.xlabel('Distance Base-Antenna 2 (cm)')
plt.ylabel('Relative error (cm)')
plt.title('Relative error on the position of the antenna 1 during experiment 1')